
# Watershed Dynamics Model 

<img src="https://www.washington.edu/brand/files/2014/09/W-Logo_Purple_Hex.png" style="float:right;width:300px;padding:20px">   

Explore recharge and landsliding in the North Cascades National Park from a HydroShare Observatory. <br />
<br />
This Jupyter Notebook runs the landslide component on a synthetic<br /> 
Landlab grid using four recharge options.<br />
<br /> 
* Setup synthetic grid and generate data<br />
* Parameterize recharge options<br /> 
* Run LandslideProbability function from Landlab landslide component<br /> 


## HydroShare Setup and Preparation

To run this notebook, we must import several libaries.
The hs_utils library provides functions for interacting with HydroShare, including resource querying, dowloading and creation. 

In [1]:
import os
from utilities import hydroshare #, observatory_localimport_timeseries
from datetime import datetime, timedelta
import pytz
import matplotlib.pyplot as plt
import seaborn
import numpy as np
import pandas as pd
import shutil

# set variables for interacting with HydroShare from this notebook
hs=hydroshare.hydroshare()

Adding the following system variables:
   HS_USR_NAME = RStrauch
   HS_RES_ID = 0f4efd1cedb64a5a9fa90cf1f248e22f
   HS_RES_TYPE = genericresource
   JUPYTER_HUB_IP = hydroroger.ncsa.illinois.edu

These can be accessed using the following command: 
   os.environ[key]

   (e.g.)
   os.environ["HS_USR_NAME"]  => RStrauch


In [4]:
# Import libraries
import landlab_landslide
from landlab_landslide import LandslideProbability  #customized landlab landslide component updating ver1 in Landlab
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib as mpl
from landlab import RasterModelGrid

If you are curious about where the data is being downloaded, click on the Jupyter Notebook dashboard icon to return to the File System view.  The homedir directory location printed above is where you can find the data and contents you will download to a HydroShare JupyterHub server.  At the end of this work session, you can migrate this data to the HydroShare iRods server as a Generic Resource. 

## Create a grid on which to calculate landslide probability

In [6]:
#Instatiate a synthetic model domain
grid = RasterModelGrid((5, 4), spacing=(0.2, 0.2))

In [7]:
# Add required fields for the landslide component.
gridnodes = grid.number_of_nodes
grid['node']['topographic__slope'] = np.random.rand(gridnodes)
scatter_dat = np.random.randint(1, 10, gridnodes)
grid['node']['topographic__specific_contributing_area']= \
         np.sort(np.random.randint(30, 900, gridnodes))
grid['node']['soil__transmissivity']= \
         np.sort(np.random.randint(5, 20, gridnodes),-1)
grid['node']['soil__mode_total_cohesion']= \
         np.sort(np.random.randint(30, 900, gridnodes))
grid['node']['soil__minimum_total_cohesion']= \
         grid.at_node['soil__mode_total_cohesion'] - scatter_dat
grid['node']['soil__maximum_total_cohesion']= \
         grid.at_node['soil__mode_total_cohesion'] + scatter_dat
grid['node']['soil__internal_friction_angle']= \
         np.sort(np.random.randint(26, 40, gridnodes))
grid['node']['soil__thickness']= \
         np.sort(np.random.randint(1, 10, gridnodes))
grid['node']['soil__density']= \
         2000. * np.ones(grid.number_of_nodes)
         
grid_size = grid.number_of_nodes

## Number of iterations to run Monte Carlo simulation

In [8]:
n = 10

 ## 1. Recharge Option - uniform distribution 

Describe


In [9]:
distribution1 = 'uniform'
Remin_value = 20.
Remax_value = 120.
LS_prob1 = LandslideProbability(grid,number_of_iterations=n,
    groudwater__recharge_distribution=distribution1,
    groundwater__recharge_min_value=Remin_value,
    groundwater__recharge_max_value=Remax_value)

## 2. Recharge Option - Lognormal-Uniform

In [10]:
distribution2 = 'lognormal_uniform'
Remean = 4.
Restandard_deviation = 0.25
LS_prob2 = LandslideProbability(grid,number_of_iterations=n,
    groudwater__recharge_distribution=distribution2,
    groundwater__recharge_mean=Remean,
    groundwater__recharge_standard_deviation=Restandard_deviation)

## 3. Recharge Option - Lognormal - Spatial

In [11]:
distribution3 = 'lognormal_spatial'
Remean3 = np.random.randint(2,7,grid_size)
Restandard_deviation3 = np.random.rand(grid_size)
LS_prob3 = LandslideProbability(grid,number_of_iterations=n,
    groudwater__recharge_distribution=distribution3,
    groundwater__recharge_mean=Remean3,
    groundwater__recharge_standard_deviation=Restandard_deviation3)

## 4. Recharge Option - Data driven - Spatial 

In [12]:
HSD_dict = {}
HSD_id_dict = {}
fract_dict = {}
# HSD id and recharge array
for vkey in range(2,8):
    HSD_dict[vkey] = np.random.randint(20,120,10)
# node id and HSD grid ids
for ckey in grid.core_nodes:
    HSD_id_dict[ckey] = np.random.randint(2,8,2)
# node id and HSD grid fraction
for ckey in grid.core_nodes:
    fract_dict[ckey] =  np.random.rand(2)
distribution4 = 'fully_distributed'
#HSD ('hydrology source domain') dictionaries
HSD_inputs = [HSD_dict,HSD_id_dict, fract_dict]

LS_prob4 = LandslideProbability(grid,number_of_iterations=n,
    groudwater__recharge_distribution=distribution4,
    groudwater__recharge_HSD_inputs=HSD_inputs)

## Run and compare the Landslide Component sensitivity based on four recharge options

Describe outputs

In [13]:

LS_prob1.calculate_landslide_probability()

LS_prob2.calculate_landslide_probability()

LS_prob3.calculate_landslide_probability()

LS_prob4.calculate_landslide_probability()


In [14]:
grid.number_of_nodes

20

In [15]:
grid.nodes

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [12, 13, 14, 15],
       [16, 17, 18, 19]])

In [19]:
LS_prob4.landslide__factor_of_safety_histogram[core_nodes[5]]

array([ 0.3832839 ,  0.40095853,  0.5536129 ,  0.43240317,  0.4274691 ,
        0.41036259,  0.42995855,  0.437291  ,  0.44057258,  0.4551912 ])

In [17]:
core_nodes = LS_prob1.grid.core_nodes

In [20]:
grid.core_nodes

array([ 5,  6,  9, 10, 13, 14])